In [45]:
import ssl
import pandas as pd
import wandb

ssl._create_default_https_context = ssl._create_unverified_context
import os
from torch.utils.data import DataLoader
from torch import nn
from torchvision.io import read_image
from torchvision.transforms import v2
import torch
import torchvision.transforms as T
import json
import urllib
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split

In [46]:
dataframe = pd.DataFrame()
dataframe['image1'] = [f'image 1 row {i}' for i in range(20)]
dataframe['image2'] = [f'image 2 row {i}' for i in range(20)]
dataframe['image3'] = [f'image 3 row {i}' for i in range(20)]
dataframe['image4'] = [f'image 4 row {i}' for i in range(20)]
dataframe['image5'] = [f'image 5 row {i}' for i in range(20)]
dataframe['target'] = [f'target 5 row {i}' for i in range(20)]
dataframe

,image1,image2,image3,image4,image5,target
0,image 1 row 0,image 2 row 0,image 3 row 0,image 4 row 0,image 5 row 0,target 5 row 0
1,image 1 row 1,image 2 row 1,image 3 row 1,image 4 row 1,image 5 row 1,target 5 row 1
2,image 1 row 2,image 2 row 2,image 3 row 2,image 4 row 2,image 5 row 2,target 5 row 2
3,image 1 row 3,image 2 row 3,image 3 row 3,image 4 row 3,image 5 row 3,target 5 row 3
4,image 1 row 4,image 2 row 4,image 3 row 4,image 4 row 4,image 5 row 4,target 5 row 4
5,image 1 row 5,image 2 row 5,image 3 row 5,image 4 row 5,image 5 row 5,target 5 row 5
6,image 1 row 6,image 2 row 6,image 3 row 6,image 4 row 6,image 5 row 6,target 5 row 6
7,image 1 row 7,image 2 row 7,image 3 row 7,image 4 row 7,image 5 row 7,target 5 row 7
8,image 1 row 8,image 2 row 8,image 3 row 8,image 4 row 8,image 5 row 8,target 5 row 8
9,image 1 row 9,image 2 row 9,image 3 row 9,image 4 row 9,image 5 row 9,target 5 row 9


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Input

In [48]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_frame):
        self.df = data_frame

    def __getitem__(self, index):
        outfit = self.df.iloc[index]
        # print(f'accessoire shape: {img_accessoire.shape}; path={outfit["Accessoire_imagePath"]}')
        # print(f'innerwear shape: {img_innerwear.shape}; path={outfit["Innerwear_imagePath"]}')
        return torch.cat([
            torch.tensor(range(index * 100 + 0 + 1, index * 100 + 0 + 11)),
            torch.tensor(range(index * 100 + 10 + 1, index * 100 + 10 + 11)),
            torch.tensor(range(index * 100 + 20 + 1, index * 100 + 20 + 11)),
            torch.tensor(range(index * 100 + 30 + 1, index * 100 + 30 + 11)),
            torch.tensor(range(index * 100 + 40 + 1, index * 100 + 40 + 11)),
        ]), outfit['target']

    def __len__(self):
        return self.df.shape[0]

In [49]:
dataloader = torch.utils.data.DataLoader(CustomDataset(dataframe), batch_size=7,
                                         shuffle=True, num_workers=0)
for batch, (X, y) in enumerate(dataloader):
    print(X.shape)
    number_of_rows = X.shape[0]
    print(X.view(number_of_rows * 5, 10))
    break


torch.Size([7, 50])
tensor([[ 101,  102,  103,  104,  105,  106,  107,  108,  109,  110],
        [ 111,  112,  113,  114,  115,  116,  117,  118,  119,  120],
        [ 121,  122,  123,  124,  125,  126,  127,  128,  129,  130],
        [ 131,  132,  133,  134,  135,  136,  137,  138,  139,  140],
        [ 141,  142,  143,  144,  145,  146,  147,  148,  149,  150],
        [1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810],
        [1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820],
        [1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830],
        [1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840],
        [1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850],
        [1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210],
        [1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220],
        [1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230],
        [1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240],


# Output

In [50]:
batch_features = torch.tensor(
    [
        [outfit_id * 1000 + clothing_item_id * 100 + x for x in range(1, 11)]  for outfit_id in range(1, 5) for clothing_item_id in range(1,6)
    ]
)

print(batch_features.shape)
batch_features


torch.Size([20, 10])


tensor([[1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110],
        [1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210],
        [1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310],
        [1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410],
        [1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510],
        [2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110],
        [2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210],
        [2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310],
        [2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 2410],
        [2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510],
        [3101, 3102, 3103, 3104, 3105, 3106, 3107, 3108, 3109, 3110],
        [3201, 3202, 3203, 3204, 3205, 3206, 3207, 3208, 3209, 3210],
        [3301, 3302, 3303, 3304, 3305, 3306, 3307, 3308, 3309, 3310],
        [3401, 3402, 3403, 3404, 3405, 3406, 3407, 3408, 3409, 3410],
        [3501, 3502,

In [51]:
batch_features =  torch.reshape(batch_features, (int(batch_features.shape[0] / 5), 50))
batch_features

tensor([[1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1201, 1202,
         1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1301, 1302, 1303, 1304,
         1305, 1306, 1307, 1308, 1309, 1310, 1401, 1402, 1403, 1404, 1405, 1406,
         1407, 1408, 1409, 1410, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508,
         1509, 1510],
        [2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2201, 2202,
         2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2301, 2302, 2303, 2304,
         2305, 2306, 2307, 2308, 2309, 2310, 2401, 2402, 2403, 2404, 2405, 2406,
         2407, 2408, 2409, 2410, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508,
         2509, 2510],
        [3101, 3102, 3103, 3104, 3105, 3106, 3107, 3108, 3109, 3110, 3201, 3202,
         3203, 3204, 3205, 3206, 3207, 3208, 3209, 3210, 3301, 3302, 3303, 3304,
         3305, 3306, 3307, 3308, 3309, 3310, 3401, 3402, 3403, 3404, 3405, 3406,
         3407, 3408, 3409, 3410, 3501, 3502, 3503, 3504, 3505, 35

In [1]:
import torch
import numpy as np


a = np.array([0.6, 0.4, 0.7])




a = list(map(lambda x: [1 - x, x] , a))
a

[[0.4, 0.6], [0.6, 0.4], [0.30000000000000004, 0.7]]